<a href="https://colab.research.google.com/github/ridhayusra/TemuKembaliCitra/blob/main/CIBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import argparse
import numpy as np
import cv2 as cv
import os
import sys
import matplotlib.pyplot as plt
from argparse import ArgumentParser
from pip._vendor.distlib.compat import raw_input
import textwrap

In [3]:

# A function to get all image paths
# in a specific  folder
def get_image_paths(root_path):
    files = []
    supported_extensions = ".bmp" ".pbm" ".pgm" ".ppm" ".jpeg" ".jpg" ".jpe" ".png" ".tiff" ".tif"
    # r=root, d=directories, f = files
    for r, d, f in os.walk(root_path):
        for file in f:
            extension = file.format().split('.')[-1]
            if extension in supported_extensions:
                files.append(os.path.join(r, file))
    return files


# Gets dataset and query image path
# from the user using an argument parser
def get_inputs_from_user():
    program = '''
              IMPORT: You have to pass the required parameters to start the program.
              Try 'main.py --help' for more information.\n              
              '''

    usage = "Extracts query image's a feature and retrieves similar image from the image database."

    parser = ArgumentParser(prog=textwrap.dedent(program),
                            usage=usage,
                            formatter_class=argparse.RawTextHelpFormatter)

    parser.add_argument("-d", "--dataset",
                        dest="dataset_path",
                        help="the path of image database",
                        metavar="DATASET",
                        required=True, type=str, nargs=1)

    parser.add_argument("-c", "--count", dest="count",
                        help="the number of the max similar images",
                        metavar="COUNT",
                        required=True, type=int, nargs=1)
    args = parser.parse_args()

    dataset_path = "".join(args.dataset_path)
    count = int(''.join(str(i) for i in args.count))

    return dataset_path, count


# Gets query image path
def get_query_image_path_from_user():
    dataset_path = raw_input("\nquery image path: ")
    return dataset_path


# A function to normalize a histogram
def normalize_histogram(hist, total_pixel_count):
    for i in range(0, hist.size, 1):
        hist[i] = hist[i] / total_pixel_count
    return hist


# A function calculates the histogram of the given image
# returns its 3-channel histogram array
def rgb_histogram(image):
    red_channel_histogram = np.zeros(256, dtype=float)
    green_channel_histogram = np.zeros(256, dtype=float)
    blue_channel_histogram = np.zeros(256, dtype=float)

    for i in range(0, image.shape[0], 1):
        for j in range(0, image.shape[1], 1):
            pixel = image[i][j]
            blue_channel_histogram[pixel[0]] += 1
            green_channel_histogram[pixel[1]] += 1
            red_channel_histogram[pixel[2]] += 1

    hist = np.array([red_channel_histogram, green_channel_histogram, blue_channel_histogram])
    return hist


# A function returns LBP histogram of the given grayscale image
def lbp_histogram(grayscale):
    hist = np.zeros(256, dtype=float)
    for i in range(0, grayscale.shape[0], 1):
        for j in range(0, grayscale.shape[1], 1):
            hist[grayscale[i][j]] += 1

    return hist


# A function to convert a RGB image to grayscale
def rgb_to_grayscale(rgb_image):
    grayscale_image = np.zeros_like(rgb_image)
    grayscale_image.astype(np.uint8)
    for i in range(0, rgb_image.shape[0], 1):
        for j in range(0, rgb_image.shape[1], 1):
            pixel = rgb_image[i][j]
            b = pixel[0]
            g = pixel[1]
            r = pixel[2]

            grayscale_image[i][j] = int(r * 0.30 + b * 0.11 + g * 0.59)

    return grayscale_image


# A function to get LBP image of the given image
def get_lbp_image(image):
    height, width, channel = image.shape
    if channel > 1:
        image = rgb_to_grayscale(image)

    lbp_image = np.zeros_like(image)
    neighbor = 3
    factor = [[1, 2, 4],
              [128, 0, 8],
              [64, 32, 16]]

    for i in range(0, height - neighbor, 1):
        for j in range(0, width - neighbor, 1):
            img = image[i:i + neighbor, j:j + neighbor]
            center = img[1][1]

            img = (img >= center) * 1
            img = img * factor
            total = np.sum(img)
            lbp_image[i + 1][j + 1] = total

    return lbp_image


# A function to print image filenames
# after finding similar images
def print_filenames(dataset, indexes, count):
    i = 1
    while i <= indexes.size and i <= count:
        print(dataset[indexes[-i]].get_filename())
        i += 1


def finding_similarity_percentage(dataset, query_image_filename, indexes, max):
    count = 0.0
    i = 1
    while i <= indexes.size and i <= max:
        filename = dataset[indexes[-i]].get_filename()
        if query_image_filename[0] == filename[0] and query_image_filename[1] == filename[1]:
            count += 1.0
        i += 1

    if indexes.size > max:
        percentage = (count / max) * 100
    else:
        percentage = (count / indexes.size) * 100

    return percentage


# A function to find the most similar images to the given image
# according to RGB values by using Manhattan City Block (L1 norm) method
def find_similar_rgb_images(dataset, size, query_image):
    index = []
    min_distance = sys.maxsize

    query_rgb_hist = query_image.get_rgb_histogram()

    for i in range(0, size, 1):
        rgb_hist = dataset[i].get_rgb_histogram()

        red_distance = hist_difference(rgb_hist[0], query_rgb_hist[0])
        blue_distance = hist_difference(rgb_hist[1], query_rgb_hist[1])
        green_distance = hist_difference(rgb_hist[2], query_rgb_hist[2])

        total_rgb_distance = red_distance + blue_distance + green_distance
        if total_rgb_distance < min_distance:
            min_distance = total_rgb_distance
            index.append(i)

    index = np.array(index)
    return index


# A function to find the most similar images to the given image
# according to LBP values by using Manhattan City Block (L1 norm) method
def find_similar_lbp_images(dataset, size, query_image):
    index = []
    min_distance = sys.maxsize

    query_lbp_hist = query_image.get_lbp_histogram()

    for i in range(0, size, 1):
        lbp_hist = dataset[i].get_lbp_histogram()
        lbp_distance = hist_difference(lbp_hist, query_lbp_hist)

        if lbp_distance < min_distance:
            min_distance = lbp_distance
            index.append(i)

    index = np.array(index)
    return index


# A function to find the most similar images to the given image
# according to the summation of LBP and RGB distances
def find_similar_images(dataset, size, query_image):
    index = []
    min_distance = sys.maxsize

    query_lbp_hist = query_image.get_lbp_histogram()
    query_rgb_hist = query_image.get_rgb_histogram()

    for i in range(0, size, 1):
        lbp_hist = dataset[i].get_lbp_histogram()
        rgb_hist = dataset[i].get_rgb_histogram()

        lbp_distance = hist_difference(lbp_hist, query_lbp_hist)
        r_distance = hist_difference(rgb_hist[0], query_rgb_hist[0])
        g_distance = hist_difference(rgb_hist[1], query_rgb_hist[1])
        b_distance = hist_difference(rgb_hist[2], query_rgb_hist[2])

        total_distance = lbp_distance + r_distance + g_distance + b_distance

        if total_distance < min_distance:
            min_distance = total_distance
            index.append(i)

    index = np.array(index)
    return index


# A function to calculate the distance of two given histogram arrays
def hist_difference(hist1, hist2):
    total = 0
    if hist1.size != hist2.size:
        print("An error has occurred while calculating the histogram difference...")
    else:
        for i in range(0, hist1.size, 1):
            total += abs(hist1[i] - hist2[i])

    return total

In [4]:

class DatasetImage:
    def __init__(self, path):
        # assignment image name as its filename
        self.name = path.split(os.path.sep)[-1]

        # reading image from the given path
        self.image = cv.imread(path)
        print("{} is read...".format(self.name))

        # getting image attributes
        self.height, self.width, self.channel = self.image.shape
        self.total_pixel_count = self.height * self.width

        # creating lbp image
        self.lbp_image = get_lbp_image(self.image)
        print("a lbp image is created for {}...".format(self.name))

        # getting image histogram arrays
        self.rgb_histogram = rgb_histogram(self.image)
        print("rgb histogram array is created for {}...".format(self.name))
        self.lbp_histogram = lbp_histogram(self.lbp_image)
        print("lbp histogram array is created for {}...".format(self.name))

        # normalizing histogram arrays after creation them
        # # normalization of R histogram
        self.rgb_histogram[0] = normalize_histogram(self.rgb_histogram[0], self.total_pixel_count)
        # # normalization of G histogram
        self.rgb_histogram[1] = normalize_histogram(self.rgb_histogram[1], self.total_pixel_count)
        # # normalization of B histogram
        self.rgb_histogram[2] = normalize_histogram(self.rgb_histogram[2], self.total_pixel_count)
        # # normalization of lbp histogram
        self.lbp_histogram = normalize_histogram(self.lbp_histogram, self.total_pixel_count)
        print("rgb and lbp histograms of {} are normalized...".format(self.name))

    def get_filename(self):
        return self.name

    def show_image(self):
        cv.imshow(self.name, self.image)

    def show_lbp_image(self):
        cv.imshow("[LBP image of  " + self.name + " ]", self.lbp_image)

    def get_rgb_histogram(self):
        return self.rgb_histogram

    def get_lbp_image(self):
        return self.lbp_image

    def get_lbp_histogram(self):
        return self.lbp_histogram

    def get_name(self):
        return self.name

In [7]:
if __name__ == '__main__.py':
    # Getting dataset and query path
    dataset_path, count = get_inputs_from_user()

    # Image paths in the dataset
    files = get_image_paths(dataset_path)
    files = np.array(files)

    # Creating dataset
    dataset = []
    for i in range(0, files.size, 1):
        print("{}...".format(i+1))
        dataset_image = DatasetImage(files[i])
        dataset.append(dataset_image)
        print("{} is added to dataset...\n".format(dataset_image.get_name()))

    while 1:

        # Getting query image path from the user
        query_image_path = get_query_image_path_from_user()

        # checking whether the given path is valid or not
        if os.path.exists(query_image_path) and os.path.isfile(query_image_path):

            # Creating dataset-image instance for query image
            query_image = DatasetImage(query_image_path)

            # Finding the most similar images
            similar_rgb_indexes = find_similar_rgb_images(dataset, files.size, query_image)
            similar_lbp_indexes = find_similar_lbp_images(dataset, files.size, query_image)
            similar_indexes = find_similar_images(dataset, files.size, query_image)

            print("\n-------------------------------------------")
            print("\nThe closest images to \"{}\" according to only RGB distances (The closest image printed first).".format(query_image.get_filename()))
            print_filenames(dataset, similar_rgb_indexes, count)
            similarity_percentage_rgb = finding_similarity_percentage(dataset, query_image.get_filename(), similar_rgb_indexes, count)
            print("The similarity percentage is %{:0.3f}".format(similarity_percentage_rgb))

            print("\nThe closest images to \"{}\" according to only LBP distances (The closest image printed first).".format(query_image.get_filename()))
            print_filenames(dataset, similar_lbp_indexes, count)
            similarity_percentage_lbp = finding_similarity_percentage(dataset, query_image.get_filename(),similar_lbp_indexes, count)
            print("The similarity percentage is %{:0.3f}".format(similarity_percentage_lbp))

            print("\nThe closest images to \"{}\" according to both LBP and RGB (The closest image printed first).".format(query_image.get_filename()))
            print_filenames(dataset, similar_indexes, count)
            similarity_percentage_both = finding_similarity_percentage(dataset, query_image.get_filename(),similar_indexes, count)
            print("The similarity percentage is %{:0.3f}".format(similarity_percentage_both))

            dataset[similar_indexes[-1]].show_image()
            query_image.show_image()

            cv.waitKey(0)
            cv.destroyWindow(dataset[similar_indexes[-1]].get_filename())
            cv.destroyWindow(query_image.get_filename())

        else:
            print("invalid file path!")